In [1]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [2]:
year_num = 4 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [3]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

In [4]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [5]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
# station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
station_loc = station_loc.drop(['ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [6]:
# Find US Stations
station_loc.head()
us_stations = station_loc.loc[station_loc['CTRY'] == 'US', :]
us_stations_list = us_stations['USAF'].tolist()

In [7]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [8]:
# yearfile = yearfiles[-1]
# print(yearfile)
# i=0
# tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
# print(len(tar.getmembers()[1:]))
# #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
# for member in tar.getmembers()[1:]:
#     name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
#     usaf = name_parts[0]
#     wban = int(name_parts[1])
#     if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
#         i=i+1
#         #if i%(stat_num//10) == 0: print(i)
#         f=tar.extractfile(member)
#         f=gzip.open(f, 'rb')
#         content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
#         content=preprocess_station_file_content(content)
#         df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
#         content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
#         df = df.append(content)
# tar.close()

In [9]:
# df

In [10]:
# station_loc

In [11]:
# merged_df = pd.merge(df, station_loc, how="right", on="USAF")
# merged_df

In [12]:
# merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
# merged_clean_df

In [13]:
# merged_clean_df.to_csv('merged_cleaned_dataframe.csv',index=False)

In [14]:
# day = df_day['YEARMODA'].max()
# df_day = df_day[df_day['YEARMODA']==day]

In [ ]:
for yearfile in yearfiles[:-1]:
    print(yearfile)
    i=0
    tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
    print(len(tar.getmembers()[1:]))
    #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
    for member in tar.getmembers()[1:]:
        name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
        usaf = name_parts[0]
        wban = int(name_parts[1])
        if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0 and (usaf in us_stations_list) and (usaf != 999999):
            i=i+1
            #if i%(stat_num//10) == 0: print(i)
            f=tar.extractfile(member)
            f=gzip.open(f, 'rb')
            content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
            content=preprocess_station_file_content(content)
#             df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
            df_day = df_day.append(content, ignore_index=True)
            content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
            df = df.append(content)
    tar.close()

gsod_2016.tar
12076
gsod_2017.tar
11923


In [ ]:
df_day

In [ ]:
station_loc

In [ ]:
station_loc['WBAN'] = station_loc['WBAN'].astype(float)
station_loc

In [ ]:
merged_df = pd.merge(df_day, station_loc, how="outer", on="USAF")
merged_df

In [ ]:
merged_clean_df = merged_df.loc[merged_df['MIN'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['MAX'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df['TEMP'] != 9999.9]
merged_clean_df = merged_clean_df.loc[merged_clean_df["USAF"] != "999999", :]
merged_clean_df.dropna(how='any')
us_merged_df = merged_clean_df.loc[merged_clean_df["CTRY"] == "US", :]
us_merged_df

In [ ]:
new_df=us_merged_df.dropna(how='any')

In [ ]:
new_df

In [ ]:
# us_drop_na_df = us_merged_df.dropna(how='any')
# us_drop_na_df

In [ ]:
new_df

In [ ]:
if (new_df['WBAN_x'].equals(new_df['WBAN_y'])):
    new_df = new_df.drop(['WBAN_y'], axis=1)
    new_df = new_df.rename(columns={"WBAN_x": "WBAN"})
new_df


In [ ]:
# us_drop_na_df['YEARMODA']

In [ ]:
# list(us_drop_na_df['YEARMODA'])

In [ ]:
list(new_df['YEARMODA'])

In [ ]:

new_df.dtypes

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeather
collection = db.weather
db.collection.insert_many(new_df.to_dict('records'))

In [ ]:
merged_agg_df = pd.merge(df, station_loc, how="outer", on="USAF")
merged_agg_df
merged_clean_agg_df = merged_agg_df.loc[merged_agg_df['MIN'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['MAX'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df['TEMP'] != 9999.9]
merged_clean_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["USAF"] != "999999", :]
merged_clean_agg_df.dropna(how='any')
us_merged_agg_df = merged_clean_agg_df.loc[merged_clean_agg_df["CTRY"] == "US", :]
us_merged_agg_df
new_agg_df=us_merged_agg_df.dropna(how='any')
if (new_agg_df['WBAN_x'].equals(new_agg_df['WBAN_y'])):
    new_agg_df = new_agg_df.drop(['WBAN_y'], axis=1)
    new_agg_df = new_agg_df.rename(columns={"WBAN_x": "WBAN"})
new_agg_df

In [ ]:
#Set a timestamp to the beginning of the month for aggregates
# https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o
# https://www.w3schools.com/python/python_datetime.asp
def add_aggregate_timestamp(row):
    return dt.datetime(int(row['YEAR']), int(row['MONTH']), 1)

new_agg_df['YEARMODA'] = new_agg_df.apply(lambda row: add_aggregate_timestamp(row), axis=1)
new_agg_df

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.USWeatherAgg
collection = db.weather
db.collection.insert_many(new_agg_df.to_dict('records'))